#### First let's import a few things

In [ ]:
import os
import ceci
from ceci.pipeline import Pipeline

#### For now we need to be in the top-level ceci directory to read the input yaml file

This is b/c it refers to the configuration file, which we will also need to read.

We should fix this.

In [ ]:
os.chdir(os.path.join(os.path.dirname(ceci.__file__), '..'))
pipeline = Pipeline.read('tests/test.yml')

#### Now we have some pipeline introspection

1. Printing a list of stages
2. Accessing a stage as an attribute of the pipeline
3. Printing the inputs and output connections associated to a stage
4. Printing the configuration associated to a stage

In [ ]:
pipeline.print_stages()

In [ ]:
print(pipeline.SysMapMaker)

In [ ]:
pipeline.SysMapMaker.print_io()

In [ ]:
print(repr(pipeline.WLGCSelector.config))

#### Ok, eventually this example should include most stuff, but for now let's just the pipeline

In [ ]:
status = pipeline.run()

#### Now let's do the same thing by building the pipeline with the python interface

In [ ]:
from ceci_example.example_stages import *

In [ ]:
pipe2 = Pipeline.interactive()
overall_inputs = {'DM':'./tests/inputs/dm.txt',
                  'fiducial_cosmology':'./tests/inputs/fiducial_cosmology.txt'}
inputs = overall_inputs.copy()
inputs['metacalibration'] = True
inputs['config'] = None

inputs.update(pipe2.add_stage(PZEstimationPipe.build(**inputs)))
inputs.update(pipe2.add_stage(shearMeasurementPipe.build(**inputs, apply_flag=False)))
inputs.update(pipe2.add_stage(WLGCSelector.build(**inputs, zbin_edges=[0.2, 0.3, 0.5], ra_range=[-5, 5])))
inputs.update(pipe2.add_stage(SysMapMaker.build(**inputs)))
inputs.update(pipe2.add_stage(SourceSummarizer.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCCov.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCRandoms.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCTwoPoint.build(**inputs)))
inputs.update(pipe2.add_stage(WLGCSummaryStatistic.build(**inputs)))


In [ ]:
pipe2.load_configs(overall_inputs, pipeline.run_config, pipeline.stages_config)

In [ ]:
pipe2.WLGCCov.outputs

In [ ]:
pipe2.WLGCCov.config['aliases'] = dict(covariance='covariance_copy')
pipe2.WLGCSummaryStatistic.config['aliases'] = dict(covariance='covariance_copy')

In [ ]:
pipe2.WLGCCov.find_outputs('.')

In [ ]:
pipe2.run()